# Intelligent Agents: Reflex-Based Agents for GridHunt

Student Name: [Add your name]

I have used the following AI tools: [list tools]

I understand that my submission needs to be my own work: [your initials]

## Learning Outcomes

* Apply core AI concepts by implementing the agent function for a simple and model-based reflex agents that respond to environmental percepts.
* Practice how the environment and the agent function interact.
* Analyze agent performance through controlled experiments across different environment configurations.

## Instructions

Total Points: Undergrads 10

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook.
### AI Use

Here are some guidelines that will make it easier for you:

* __Don't:__ Rely on AI auto completion. You will waste a lot of time trying to figure out how the suggested code relates to what we do in class. Turn off AI code completion (e.g., Copilot) in your IDE.
* __Don't:__ Do not submit code/text that you do not understand or have not checked to make sure that it is complete and correct.
* __Do:__ Use AI for debugging and letting it explain code and concepts from class.

## Introduction

![GridHunt Image](https://mhahsler.github.io/CS7320-AI/Agents/gridhunt.png)

Gridhunt-Agent is a small educational game to play with implementing simple agent functions. It is a modified Python reimplementation based on [Gridhunt2](https://michael.hahsler.net/SMU/CS1342/gridhunt2/). 

The objective of gridhunt is to implement a hunter who can catch the monster faster than all other hunters. Gridhunt uses a $n \times n$ grid of squares and a monster moves on this grid. Gridhunt is turn-based. A hunter catches the monster if she/he moves on the same square the monster currently occupies first. If the monster survives `max_step` rounds, the monster wins.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit (i.e., a step). The performance is measured as the sum of the energy units (steps) used to catch the monster. Catching the monster means to be on the same square.

__Environment:__ An arena with $n \times n$ squares. At the beginning the monster and the player is randomly placed in the arena environment.

__Actuators:__ The agent can move to an adjacent square or teleport which will move the agent to a random square in the arena.

__Sensors:__ The agent can see its location in the arena and the location of the monster.  

## The Arena Environment

The environment manages the location of the agents (hunter and monster). It initially places them in a random location and then provides them in every step with percepts and asks them for their action. 

__A Note on positions:__
The arena is an array with row and column indices representing the position.
Positions are stored in a numpy array where `pos[0]` is the row index and `pos[1]` is the column index in the arena. Remember indices in Python start with 0 and go to `n-1`.

In [1]:
import numpy as np
from IPython.display import clear_output
from time import sleep

def arena_environment(hunter_agent_function, monster_agent_function, n = 30, max_steps = 100, visualize = False, animation = False):
    """
    Simulate an arena where a hunter agent tries to catch a monster agent.

    Parameters:
    hunter_agent_function (function): A function that takes the current positions of the hunter and monster
                                      and returns the next move for the hunter ('north', 'east', 'west', 'south', 'teleport').
    monster_agent_function (function): A function that takes the current positions of the hunter and monster
                                       and returns the next move for the monster (or 'stay' to remain in place).
    n (int): The size of the arena (n x n grid).
    max_steps (int): The maximum number of steps to simulate.
    visualize (bool): Whether to visualize the arena.
    animation (bool): Whether to animate the visualization with a delay.

    Returns:
    int: The number of steps taken for the hunter to catch the monster or np.nan (not a number) if not caught.
    """
    
    # Initialize positions
    monster_pos = np.random.randint(0, n-1, size=2)
    hunter_pos = np.random.randint(0, n-1, size=2)
    
    def move(action, position):
        """calculate new position for the agent."""
        if action == 'north':
            position[0] -= 1
        elif action == 'south':
            position[0] += 1
        elif action == 'west':
            position[1] -= 1
        elif action == 'east':
            position[1] += 1
        else:
            raise ValueError("Invalid action.")
        
        # Ensure position stays within bounds
        position = np.clip(position, 0, n-1)
        return position

    def print_arena():

        if animation:
            sleep(1)
            clear_output(wait=True)
        print(f"Step {step}:")
        print(f"Hunter is at '{hunter_pos}'; Monster is at '{monster_pos}'\n")
        arena = np.full((n, n), '.', dtype=str)
        arena[monster_pos[0], monster_pos[1]] = 'M'
        arena[hunter_pos[0], hunter_pos[1]] = 'H'
        print("\n".join("".join(row) for row in arena))

    for step in range(max_steps):
        if visualize:
            print_arena()
        
        # Check if hunter has caught the monster
        if np.array_equal(hunter_pos, monster_pos):
            if visualize:
                print(f"Hunter caught the monster in {step} steps!")
            return step
        
        # Get next move from monster agent function
        monster_action = monster_agent_function(hunter_pos, monster_pos)
        if monster_action != 'stay':
            monster_pos = move(monster_action, monster_pos)

        # Get next move from hunter agent function
        hunter_action = hunter_agent_function(hunter_pos, monster_pos)
        if hunter_action == 'teleport':
            hunter_pos = np.random.randint(0, n-1, size=2)
        else:
            hunter_pos = move(hunter_action, hunter_pos)
    
        # print the agents' actions
        if visualize:
            print(f"Hunter chose action '{hunter_action}'")
            print(f"Monster chose action '{monster_action}'")
            print("\n" + "="*n + "\n")

    # the hunter failed to catch the monster within max_steps
    if visualize:
        print(f"Hunter failed to catch the monster in {max_steps} steps.")
    
    return np.nan

I implement the monster here as a simple agent that moves around randomly, but mostly stays in its place.

In [2]:
monster_actions = ["north", "east", "west", "south", "stay"]

def monster_agent_function_simple(hunter_pos, monster_pos):
    return np.random.choice(monster_actions, p=[0.125, 0.125, 0.125, 0.125, 0.5])

# The Hunter Agent

Your job is to implement a simple-reflex hunter agent that can catch the monster. Remember, your agent is implemented as an agent function that get percepts and needs to return a valid action. The actions are: 

In [3]:
actions = ["north", "east", "west", "south", "teleport"]

## A Simple Implementation

Here is a very simple hunter that just runs around randomly and hopes that it bumps into the Monster. 

In [4]:
def simple_randomized_hunter_agent_function(hunter_location, monster_location):
    return np.random.choice(actions)

## Experimenting With the Agent

We can place the monster and the hunter into the environment and run a simulation by calling the environment function.

In [5]:
arena_environment(simple_randomized_hunter_agent_function, monster_agent_function_simple, n=5, max_steps=5, visualize=True, animation=False)

Step 0:
Hunter is at '[0 2]'; Monster is at '[1 3]'

..H..
...M.
.....
.....
.....
Hunter chose action 'east'
Monster chose action 'stay'

=====

Step 1:
Hunter is at '[0 3]'; Monster is at '[1 3]'

...H.
...M.
.....
.....
.....
Hunter chose action 'west'
Monster chose action 'north'

=====

Step 2:
Hunter is at '[0 2]'; Monster is at '[0 3]'

..HM.
.....
.....
.....
.....
Hunter chose action 'east'
Monster chose action 'stay'

=====

Step 3:
Hunter is at '[0 3]'; Monster is at '[0 3]'

...H.
.....
.....
.....
.....
Hunter caught the monster in 3 steps!


3

Well, this was one run, maybe it was bad luck!

Let's run an experiment with 100 runs in a $10 \times 10$ arena.  

In [6]:
steps = [arena_environment(simple_randomized_hunter_agent_function, monster_agent_function_simple, n=10, max_steps=100, visualize=False) for _ in range(100)] 
print(steps)

print (f"Hunter failed {np.sum(np.isnan(steps))} times.")
print (f"Average steps to catch the monster over successful runs: {round(np.nanmean(steps), 2)}")

[32, 75, 7, nan, 41, nan, nan, 68, nan, 80, nan, nan, 59, 77, nan, 18, nan, 45, nan, nan, nan, 81, 60, 18, nan, nan, nan, 11, nan, 76, nan, nan, nan, nan, nan, 7, 2, 79, 41, nan, 59, nan, nan, 3, 0, nan, 13, nan, 1, 67, nan, 2, nan, nan, nan, nan, nan, nan, nan, 25, 14, 50, 99, 52, nan, 27, nan, 67, 63, 16, nan, 16, 80, 5, 73, nan, nan, 84, 55, nan, 91, 33, 39, 63, nan, nan, nan, 5, nan, 16, 28, 13, 41, 62, 13, nan, nan, 15, 57, 8]
Hunter failed 45 times.
Average steps to catch the monster over successful runs: 40.58


This in not a very good agent. You need to implement a better agent function.

# Your Agent Implementation

Write a new hunter agent function that chases the monster. Copy the simple randomized hunter function from above and modify it using rules based on the monster and hunter location.

In [7]:
# here goes your agent implementation

# Your Experiments

Copy the simulation code from above and run experiments with your agent. 
Experiment with larger arenas of at least size $30 \times 30$.

In [8]:
# Your experimentation code goes here

## Your Conclusion

Discuss what your final strategy is and how the arena size affects your hunter agent's performance.



> Your discussion goes here

The monster is also an agent. Describe how the monster's agent function could be changed so it gets better at avoiding the hunter.

> Your discussion goes here

# More Work (Optional)

Here are some ideas:

* Implement a better monster agent function and perform experiments
* Change the environment so multiple hunters can hunt the monster.
* Give the hunter a new action that shoots an arrow in a specific direction. The arrow can go a maximum of 5 squares. 
  If the hunter hits the monster, then it wins.
